In [1]:
import pandas as pd
import numpy as np
import torch

df = pd.read_csv('./csv_dataset/cropped.csv')
df.dropna()
print(df.head())

   age  gender ethnicity                               img_name
0    2       1         2   2_1_2_20161219151919100.jpg.chip.jpg
1    7       0         0   7_0_0_20170110215633251.jpg.chip.jpg
2   56       0         0  56_0_0_20170111200016066.jpg.chip.jpg
3   53       0         0  53_0_0_20170105172607885.jpg.chip.jpg
4   45       0         2  45_0_2_20170105173303117.jpg.chip.jpg


In [2]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(df, train_size=0.8, random_state=42)
df_train, df_valid = train_test_split(df_train, train_size=0.85, random_state=42)

# Save the training, validation, and test sets in separate CSV files.
df_train.to_csv('./csv_dataset/train_set.csv', index=False)
df_valid.to_csv('./csv_dataset/valid_set.csv', index=False)
df_test.to_csv('./csv_dataset/test_set.csv', index=False)

In [3]:
from torchvision import transforms
custom_transform = transforms.Compose([transforms.Resize((128, 128)),
                                       transforms.RandomCrop((120, 120)),
                                       transforms.ToTensor()])


In [4]:
# Hyper-parameters
import torch.cuda
learning_rate = 0.0005
device = 'cuda' if torch.cuda.is_available() else 'cpu'
num_epochs = 30
train_batch_size = 128
test_batch_size = 128
random_seed = 1
NUM_CLASSES = 117
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [8]:
from torch.utils.data import Dataset, DataLoader
from pathlib import Path
from PIL import Image

class UTKface(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data = pd.read_csv(csv_file)
        self.transform = transform
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        row = self.data.iloc[index].values
        img_dir = str(Path(f'./crop_part1/{row[-1]}'))
        image = Image.open(img_dir).convert("L")
        image = self.transform(image)
        age = torch.tensor(int(row[0]))
        
        return image, age
    
    
csv_file_train = './csv_dataset/train_set.csv'
csv_file_test = './csv_dataset/test_set.csv'
csv_file_valid = './csv_dataset/valid_set.csv'


train_set = UTKface(csv_file_train,custom_transform)
test_set = UTKface(csv_file_test,custom_transform)
valid_set = UTKface(csv_file_valid,custom_transform)

train_loader = DataLoader(train_set, batch_size=train_batch_size, shuffle=True)
valid_loader = DataLoader(valid_set, batch_size=test_batch_size)
test_loader = DataLoader(test_set, batch_size=test_batch_size)


for images, age in train_loader:  
    print('Image batch dimensions:', images.shape)
    print('Image label dimensions:', age.shape)
    break
        

Image batch dimensions: torch.Size([128, 1, 120, 120])
Image label dimensions: torch.Size([128])


In [10]:
from coral_pytorch.layers import CoralLayer

class ConvNet(torch.nn.Module):

    def __init__(self, num_classes):
        super(ConvNet, self).__init__()

        self.features = torch.nn.Sequential(
            torch.nn.Conv2d(1, 3, (3, 3), (1, 1), 1),
            torch.nn.MaxPool2d((2, 2), (2, 2)),
            torch.nn.Conv2d(3, 6, (3, 3), (1, 1), 1),
            torch.nn.MaxPool2d((2, 2), (2, 2)))

        ### Specify CORAL layer
        self.fc = CoralLayer(size_in=6144, num_classes=num_classes)
        ###--------------------------------------------------------------------###

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1) # flatten

        ##### Use CORAL layer #####
        logits =  self.fc(x)
        probas = torch.sigmoid(logits)
        ###--------------------------------------------------------------------###

        return logits, probas



torch.manual_seed(random_seed)
model = ConvNet(num_classes=NUM_CLASSES)
model.to(DEVICE)

optimizer = torch.optim.Adam(model.parameters())

ModuleNotFoundError: No module named 'coral_pytorch'

In [9]:
from coral_pytorch.dataset import levels_from_labelbatch
from coral_pytorch.losses import coral_loss


for epoch in range(num_epochs):

    model = model.train()
    for batch_idx, (features, class_labels) in enumerate(train_loader):

        ##### Convert class labels for CORAL
        levels = levels_from_labelbatch(class_labels, 
                                        num_classes=NUM_CLASSES)
        ###--------------------------------------------------------------------###

        features = features.to(DEVICE)
        levels = levels.to(DEVICE)
        logits, probas = model(features)

        #### CORAL loss 
        loss = coral_loss(logits, levels)
        ###--------------------------------------------------------------------###   


        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        ### LOGGING
        if not batch_idx % 200:
            print ('Epoch: %03d/%03d | Batch %03d/%03d | Loss: %.4f' 
                   %(epoch+1, num_epochs, batch_idx, 
                     len(train_loader), loss))

ModuleNotFoundError: No module named 'coral_pytorch'

In [ ]:
from coral_pytorch.dataset import proba_to_label


def compute_mae_and_mse(model, data_loader, device):

    with torch.no_grad():

        mae, mse, acc, num_examples = 0., 0., 0., 0

        for i, (features, targets) in enumerate(data_loader):

            features = features.to(device)
            targets = targets.float().to(device)

            logits, probas = model(features)
            predicted_labels = proba_to_label(probas).float()

            num_examples += targets.size(0)
            mae += torch.sum(torch.abs(predicted_labels - targets))
            mse += torch.sum((predicted_labels - targets)**2)

        mae = mae / num_examples
        mse = mse / num_examples
        return mae, mse

In [ ]:
train_mae, train_mse = compute_mae_and_mse(model, train_loader, DEVICE)
test_mae, test_mse = compute_mae_and_mse(model, test_loader, DEVICE)

NameError: name 'compute_mae_and_mse' is not defined

In [ ]:
print(f'Mean absolute error (train/test): {train_mae:.2f} | {test_mae:.2f}')
print(f'Mean squared error (train/test): {train_mse:.2f} | {test_mse:.2f}')
